In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pickle
import itertools

In [7]:
# 刷新operators檔案
import importlib
import operators

importlib.reload(operators)
from operators import *

# 資料輸入與清理

In [3]:
# 價量資料
with open(r"./Input_data/pv_data_dict.pkl", "rb") as f:
    pv_data_dict = pickle.load(f)
print(list(pv_data_dict.keys()))

# 財報資料
with open(r"./Input_data/fs_data_dict.pkl", "rb") as f:
    fs_data_dict = pickle.load(f)
print(list(fs_data_dict.keys()))

# 籌碼資料
with open(r"./Input_data/flow_data_dict.pkl", "rb") as f:
    flow_data_dict = pickle.load(f)
print(list(flow_data_dict.keys()))


price_data = pd.read_pickle(r"./Input_data/price_data.pkl")

['open', 'high', 'low', 'close', 'volume', 'value_dollars', 'daily_ret', 'cap', 'pe', 'pb', 'dividend_yield', 'market', 'vwap', 'ret1', 'ret2', 'ret3', 'ret4', 'ret5', 'ret6', 'ret7', 'ret8', 'ret9', 'ret10', 'ret11', 'ret12', 'ret13', 'ret14', 'ret15', 'ret16', 'ret17', 'ret18', 'ret19', 'ret20']
['net_sales', 'cogs', 'gross_margin', 'op_expense', 'op_income', 'eps', 'net_income', 'ebit', 'ebitda', 'ind_code', 'cash', 'total_asset', 'total_liability', 'total_equity', 'current_asset', 'non_current_asset', 'current_liability', 'non_current_liability', 'st_borrowing', 'ar', 'inventory', 'ap', 'ppe', 'goodwill', 'cfo', 'cfi', 'cff', 'beg_cash', 'end_cash', 'fcf', 'capex', 'monthly_sales', 'sales_mom', 'sales_yoy']
['foreign_net_buy_shares', 'investment_trust_net_buy_shares', 'dealer_net_buy_shares', 'margin_balance_shares', 'margin_balance_dollars', 'foreign_holding_ratio', 'investment_trust_holding_ratio', 'dealer_holding_ratio', 'insider_holding_ratio']


# 挖因子選取範圍定義

In [4]:
def back_test(factor_df, window=5, start_date= '2014-01-01', end_date= '2020-12-31', asc= True):

    # Method 1: N個資金通道
    Top_10_bool_df = factor_df.rank(axis=1, method="min", ascending = asc) <= 10
    Top_10_bool_df = Top_10_bool_df[start_date:end_date]
    weight_df = Top_10_bool_df.div(Top_10_bool_df.sum(axis=1),axis=0)

    weight_df_hold = weight_df.rolling(window=window).mean()
    
    exp_ret_df = pv_data_dict['ret1']
    exp_ret_df = exp_ret_df[start_date:end_date]

    strategy_ret = (exp_ret_df * weight_df_hold).sum(axis=1)
    
    # 績效結算
    years = len(strategy_ret)/ 252
    sharpe = strategy_ret.mean()/ strategy_ret.std() * math.sqrt(252)

    return sharpe


In [58]:
factor_name = "MA5"
expr = 'mul(ts_max(monthly_sales, 55), cs_scale(eps))'
expr =expr.lower()
factor_df = factor_expr(expr)

print(back_test(factor_df))

0.32353136007588146


In [13]:
# Future Return df
exp_ret = pv_data_dict['ret5']

# 定義選取範圍
ts_func_list = list(ts_op_dict.keys())  # 時間序列操作
ts_pair_list = list(itertools.combinations(ts_func_list, 2))  

cs_func_list = list(cs_op_dict.keys())  # 橫截面操作
bin_func_list = list(bin_dict.keys())  # 兩兩一組操作
sing_func_list = ['abs', 'log']  # 對自己進行操作，無須給額外參數
bin_comp_func_list = ['corelation', 'covariance'] # 兩兩一組，須給定額外參數 (d)


價量資料 = list(pv_data_dict.keys())[:8]
財報資料 = list(fs_data_dict.keys())
財報資料.remove('ind_code')
籌碼資料 = list(flow_data_dict.keys())

data_list = 價量資料 + 財報資料 + 籌碼資料

print(data_list)


['open', 'high', 'low', 'close', 'volume', 'value_dollars', 'daily_ret', 'cap', 'net_sales', 'cogs', 'gross_margin', 'op_expense', 'op_income', 'eps', 'net_income', 'ebit', 'ebitda', 'cash', 'total_asset', 'total_liability', 'total_equity', 'current_asset', 'non_current_asset', 'current_liability', 'non_current_liability', 'st_borrowing', 'ar', 'inventory', 'ap', 'ppe', 'goodwill', 'cfo', 'cfi', 'cff', 'beg_cash', 'end_cash', 'fcf', 'capex', 'monthly_sales', 'sales_mom', 'sales_yoy', 'foreign_net_buy_shares', 'investment_trust_net_buy_shares', 'dealer_net_buy_shares', 'margin_balance_shares', 'margin_balance_dollars', 'foreign_holding_ratio', 'investment_trust_holding_ratio', 'dealer_holding_ratio', 'insider_holding_ratio']


In [12]:
expr_list = {}  # 紀錄表達式與對應的分數
tmp_dict = {}  # 把表達式紀錄下來，並給予編號
import Alpha
importlib.reload(Alpha)
from Alpha import alpha_dict

import 所有因子表達式
importlib.reload(所有因子表達式)
from 所有因子表達式 import alpha


In [20]:
import optuna

def objective(trial):   # 籌碼資料vs噪音
    '''cs_op = trial.suggest_categorical('cs_op',cs_func_list)
    ts_op = trial.suggest_categorical("ts_op", ts_func_list)
    ts_op2 = trial.suggest_categorical("ts_op2", ts_func_list)
    bin_op = trial.suggest_categorical("math_op", bin_func_list)
    ts_window = trial.suggest_int("ts_window", 5, 20, step=5)

    data_pair = trial.suggest_categorical("data_pair", data_pairs_str)
    data_name1, data_name2 = data_pair.split('|')
    expr = f'{ts_op2}({cs_op}({ts_op}({bin_op}({data_name2}, {data_name1}),{ts_window})), {ts_window})'''


    #data_name1 = trial.suggest_categorical("data_name1", data_list)
    #remaining_list = [d for d in data_list if d != data_name1]
    #data_name2 = trial.suggest_categorical("data_name2", remaining_list)
    chip_name = trial.suggest_categorical("chip_factor", 籌碼資料)

    # 時序平滑（流入水位）
    flow_ts_op = trial.suggest_categorical("flow_ts_op", ["ts_mean", "ts_sum", "ts_rank","ts_delta"])
    w1 = trial.suggest_int("flow_window", 5, 60, step=5)

    # 穩定度/噪音（波動）
    noise_ts_op = trial.suggest_categorical("noise_ts_op", ["ts_stddev"])
    w2 = trial.suggest_int("noise_window", 5, 60, step=5)

    # 結合方式：越大越「smart」(你可用 Optuna 自己挑)
    # 常見：flow / (noise + eps) 或 flow - k*noise
    combine = trial.suggest_categorical("combine", ["ratio", "diff"])
    eps = trial.suggest_float("eps", 1e-6, 1e-2, log=True)  # 防止除以 0
    k = trial.suggest_float("k", 0.1, 5.0, log=True)        # diff 的權重

    # 最後做 CS 讓尺度可比
    cs_norm = trial.suggest_categorical("cs_norm", ["cs_rank", "cs_zscore", "cs_scale"])

    Data1 = f"{flow_ts_op}({chip_name}, {w1})"
    Data2 = f"{noise_ts_op}({chip_name}, {w2} )"

    # combine
    if combine == "ratio":
        # flow / (noise + eps) ：穩定流入越大越好
        expr = f"{Data1}/({Data2} + {eps})" 
    elif combine == "diff":
        # flow - k*noise ：同樣流入下，噪音越小越好
        expr =  f"sub({Data1}, {k} * {Data2})" 
    
    expr = f"{cs_norm}({expr})"

    expr = expr.lower()
    factor_df = factor_expr(expr)
    ic_series =  factor_df.corrwith(exp_ret, method='spearman', axis=1) # rank IC計算
    ic_mean = ic_series.mean()
    ic_ir = ic_mean/ ic_series.std()

    if ic_mean > 0:
        ascending = False # IC小於0 (負相關)時，取因子值最小的10個
    else:
        ascending = True 
    
    sharpe = back_test(factor_df, window=5, start_date= '2015-01-01', end_date= '2020-12-31', asc = ascending)
    score = abs(100*ic_mean + 10*ic_ir) # 定義最大化機制

    if (score >= 5) and (expr not in alpha.values())  and (expr not in tmp_dict.values()) and (sharpe >= 0.6):
    #if expr not in alpha_dict.values() and sharpe > 1:
        expr_list[expr] = f"{score:.3f}, {sharpe:.3f}"
        print(f'1 factor with sharpe:{sharpe} added')
    return score

def objective_multi(trial):
    因子數量 = trial.suggest_int('因子數量', 1,2)
    if 因子數量 == 1:
        整理步驟數 = trial.suggest_int("整理步驟數", 1,3)
        因子1 = trial.suggest_categorical('因子1', data_list)
        expr = f"{因子1}"
        
        for step in range(整理步驟數):
            整理方法 = trial.suggest_categorical(f"單因子整理方法{step + 1}",
                                             ts_func_list + cs_func_list)
            if 整理方法.startswith('ts_'):
                期間參數 = trial.suggest_int(f"單因子期間參數{step + 1}", 5, 60, step= 5)
                expr = f"{整理方法}({expr}, {期間參數})"

            elif 整理方法.startswith('cs_'):
                expr = f"{整理方法}({expr})"

    elif 因子數量 == 2:
        測試方式 = trial.suggest_int('測試方式', 1, 3)
        if 測試方式 == 1:  # 兩兩ts/cs處理完再interaction method
            整理步驟數 = 2

            因子1 = trial.suggest_categorical('因子1', data_list)
            因子2 = trial.suggest_categorical('因子2', data_list)

            整理方法1 = trial.suggest_categorical("多因子整理方法1", ts_func_list + cs_func_list)
            if 整理方法1.startswith('ts_'):
                期間參數1 = trial.suggest_int(f"多因子期間參數1", 5, 60, step= 5)
                Data1 = f"{整理方法1}({因子1}, {期間參數1})"
            elif 整理方法1.startswith('cs_'):
                Data1 = f"{整理方法1}({因子1})"

            整理方法2 = trial.suggest_categorical("多因子整理方法2", ts_func_list + cs_func_list)
            if 整理方法2.startswith('ts_'):
                期間參數2 = trial.suggest_int(f"多因子期間參數2", 5, 60, step= 5)
                Data2 = f"{整理方法2}({因子2}, {期間參數2})"
            elif 整理方法2.startswith('cs_'):
                Data2 = f"{整理方法2}({因子2})"

            interaction_method1 = trial.suggest_categorical('interaction_method1', bin_func_list)
            expr = f"{interaction_method1}({Data1}, {Data2})"

        elif 測試方式 == 2:  # 倆倆先處理再套一個整理方式
            因子1 = trial.suggest_categorical('因子1', data_list)
            因子2 = trial.suggest_categorical('因子2', data_list)

            interaction_method1 = trial.suggest_categorical('interaction_method1', bin_func_list)
            expr = f"{interaction_method1}({因子1}, {因子2})"

            整理方法1 = trial.suggest_categorical("多因子整理方法1", ts_func_list + cs_func_list)
            if 整理方法1.startswith('ts_'):
                期間參數1 = trial.suggest_int(f"多因子期間參數1", 5, 60, step= 5)
                expr = f"{整理方法1}({expr}, {期間參數1})"
            elif 整理方法1.startswith('cs_'):
                expr = f"{整理方法1}({expr})"
        
        elif 測試方式 == 3:  # 自處理因子1, 兩兩處理，再套兩層ts/cs
            因子1 = trial.suggest_categorical('因子1', data_list)
            因子2 = trial.suggest_categorical('因子2', data_list)

            self_method1 = trial.suggest_categorical('self_method1', sing_func_list)
            Data1 = f"{self_method1}({因子1})"

            interaction_method1 = trial.suggest_categorical('interaction_method1', bin_func_list)
            expr = f"{interaction_method1}({Data1}, {因子2})"

            整理方法1 = trial.suggest_categorical("多因子整理方法1", ts_func_list + cs_func_list)
            if 整理方法1.startswith('ts_'):
                期間參數1 = trial.suggest_int(f"多因子期間參數1", 5, 60, step= 5)
                expr = f"{整理方法1}({expr}, {期間參數1})"
            elif 整理方法1.startswith('cs_'):
                expr = f"{整理方法1}({expr})"

            整理方法2 = trial.suggest_categorical("多因子整理方法2", ts_func_list + cs_func_list)
            if 整理方法2.startswith('ts_'):
                期間參數2 = trial.suggest_int(f"多因子期間參數2", 5, 60, step= 5)
                expr = f"{整理方法2}({expr}, {期間參數2})"
            elif 整理方法2.startswith('cs_'):
                expr = f"{整理方法2}({expr})"
            

    expr = expr.lower()
    factor_df = factor_expr(expr)
    ic_series =  factor_df.corrwith(exp_ret, method='spearman', axis=1) # rank IC計算
    ic_mean = ic_series.mean()
    ic_ir = ic_mean/ ic_series.std()

    if ic_mean > 0:
        ascending = False # IC小於0 (負相關)時，取因子值最小的10個
    else:
        ascending = True 
    
    sharpe = back_test(factor_df, window=5, start_date= '2015-01-01', end_date= '2020-12-31', asc = ascending)
    score = abs(100*ic_mean + 10*ic_ir) # 定義最大化機制

    if (score >= 5) and (expr not in alpha.values())  and (expr not in tmp_dict.values()) and (sharpe >= 0.6):
    #if expr not in alpha_dict.values() and sharpe > 1:
        expr_list[expr] = f"{score:.3f}, {sharpe:.3f}"
        print(f'1 factor with sharpe:{sharpe} added')

    return score 

In [14]:
def objective_gpt(trial):
    pattern = trial.suggest_categorical('pattern', ['fin_price_mix', 'relative_strength'])

    if pattern == 'fin_price_mix':  # 財報 + 價量混合
        財報因子 = trial.suggest_categorical('財報因子', 財報資料)
        價量因子 = trial.suggest_categorical('價量因子', 價量資料)

        時序整理1 = trial.suggest_categorical('時序整理1', ts_func_list)
        時序整理2 = trial.suggest_categorical('時序整理2', ts_func_list)

        期間參數1 = trial.suggest_int(f"多因子期間參數1", 5, 60, step= 5)
        期間參數2 = trial.suggest_int(f"多因子期間參數2", 5, 60, step= 5)
        互動方式 = trial.suggest_categorical('interaction_method', bin_func_list)

        expr = f"cs_rank({互動方式}({時序整理1}({財報因子}, {期間參數1}),{時序整理2}({價量因子}, {期間參數2})))"
    
    elif pattern == 'relative_strength':  # 雙因子相對強弱
        財報籌碼因子 = trial.suggest_categorical('財報籌碼因子', 財報資料)
        價量因子 = trial.suggest_categorical('價量因子', 價量資料)

        時序整理1 = trial.suggest_categorical('時序整理1', ts_func_list)
        時序整理2 = trial.suggest_categorical('時序整理2', ts_func_list)

        期間參數1 = trial.suggest_int(f"多因子期間參數1", 5, 60, step= 5)
        期間參數2 = trial.suggest_int(f"多因子期間參數2", 5, 60, step= 5)
        互動方式 = trial.suggest_categorical('interaction_method', bin_func_list)

        expr = f"{互動方式}(cs_rank({時序整理1}({財報籌碼因子}, {期間參數1})),cs_rank({時序整理2}({價量因子}, {期間參數2})))"
        
    
    expr = expr.lower()
    factor_df = factor_expr(expr)
    ic_series =  factor_df.corrwith(exp_ret, method='spearman', axis=1) # rank IC計算
    ic_mean = ic_series.mean()
    ic_ir = ic_mean/ ic_series.std()

    if ic_mean > 0:
        ascending = False # IC小於0 (負相關)時，取因子值最小的10個
    else:
        ascending = True 
    
    sharpe = back_test(factor_df, window=5, start_date= '2014-01-01', end_date= '2020-12-31', asc = ascending)
    score = abs(100*ic_mean + 10*ic_ir) # 定義最大化機制

    if (score >= 6) and (expr not in alpha.values())  and (expr not in expr_list.keys()) and (sharpe >= 0.7):
    #if expr not in alpha_dict.values() and sharpe > 1:
        expr_list[expr] = f"{score:.3f}, {sharpe:.3f}"
        print(f'1 factor with sharpe:{sharpe} added')

    return score 
        

In [21]:
# Creating Optuna object and defining its parameters
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials = 50)

# Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2025-12-27 09:37:59,762] A new study created in memory with name: no-name-db198801-80eb-4d79-b428-c996dfa36f97
[I 2025-12-27 09:38:02,811] Trial 0 finished with value: 2.461762734616316 and parameters: {'chip_factor': 'investment_trust_holding_ratio', 'flow_ts_op': 'ts_rank', 'flow_window': 20, 'noise_ts_op': 'ts_stddev', 'noise_window': 40, 'combine': 'ratio', 'eps': 0.0008324446923462859, 'k': 0.2786867532124058, 'cs_norm': 'cs_rank'}. Best is trial 0 with value: 2.461762734616316.
[I 2025-12-27 09:38:04,696] Trial 1 finished with value: 3.026161958279764 and parameters: {'chip_factor': 'investment_trust_net_buy_shares', 'flow_ts_op': 'ts_mean', 'flow_window': 25, 'noise_ts_op': 'ts_stddev', 'noise_window': 40, 'combine': 'ratio', 'eps': 1.2758269087678692e-05, 'k': 2.4496635882957434, 'cs_norm': 'cs_zscore'}. Best is trial 1 with value: 3.026161958279764.
[I 2025-12-27 09:38:06,355] Trial 2 finished with value: 1.5839413704304706 and parameters: {'chip_factor': 'dealer_holding_ra

1 factor with sharpe:0.7903887316126226 added


C:\Users\Sean\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\nanops.py:1632: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return spearmanr(a, b)[0]
[I 2025-12-27 09:38:40,996] Trial 20 finished with value: 3.7173131811996836 and parameters: {'chip_factor': 'insider_holding_ratio', 'flow_ts_op': 'ts_delta', 'flow_window': 15, 'noise_ts_op': 'ts_stddev', 'noise_window': 25, 'combine': 'ratio', 'eps': 4.914681304634879e-06, 'k': 0.33089731583634946, 'cs_norm': 'cs_rank'}. Best is trial 9 with value: 5.163697399950189.
[I 2025-12-27 09:38:42,944] Trial 21 finished with value: 4.799415073489078 and parameters: {'chip_factor': 'foreign_holding_ratio', 'flow_ts_op': 'ts_mean', 'flow_window': 5, 'noise_ts_op': 'ts_stddev', 'noise_window': 10, 'combine': 'ratio', 'eps': 3.927677583547583e-05, 'k': 0.26190994942008927, 'cs_norm': 'cs_rank'}. Best is trial 9 

1 factor with sharpe:1.017822316724978 added


[I 2025-12-27 09:38:47,048] Trial 23 finished with value: 5.117181820916041 and parameters: {'chip_factor': 'foreign_holding_ratio', 'flow_ts_op': 'ts_mean', 'flow_window': 20, 'noise_ts_op': 'ts_stddev', 'noise_window': 30, 'combine': 'ratio', 'eps': 0.0002028562010456259, 'k': 0.8183425142293707, 'cs_norm': 'cs_rank'}. Best is trial 9 with value: 5.163697399950189.


...

1 factor with sharpe:0.8415556549462975 added


[I 2025-12-27 09:39:30,254] Trial 44 finished with value: 2.9615641984293686 and parameters: {'chip_factor': 'investment_trust_net_buy_shares', 'flow_ts_op': 'ts_mean', 'flow_window': 25, 'noise_ts_op': 'ts_stddev', 'noise_window': 30, 'combine': 'ratio', 'eps': 7.907006157834236e-05, 'k': 0.16201223162759892, 'cs_norm': 'cs_scale'}. Best is trial 9 with value: 5.163697399950189.
[I 2025-12-27 09:39:32,114] Trial 45 finished with value: 1.937251540330268 and parameters: {'chip_factor': 'margin_balance_shares', 'flow_ts_op': 'ts_mean', 'flow_window': 20, 'noise_ts_op': 'ts_stddev', 'noise_window': 20, 'combine': 'ratio', 'eps': 0.0015031270113232572, 'k': 0.21405555568227075, 'cs_norm': 'cs_scale'}. Best is trial 9 with value: 5.163697399950189.
[I 2025-12-27 09:39:33,784] Trial 46 finished with value: 1.0126417290206837 and parameters: {'chip_factor': 'foreign_net_buy_shares', 'flow_ts_op': 'ts_mean', 'flow_window': 40, 'noise_ts_op': 'ts_stddev', 'noise_window': 40, 'combine': 'ratio'

1 factor with sharpe:0.7078170430370518 added


[I 2025-12-27 09:39:38,749] Trial 49 finished with value: 2.7283332819998445 and parameters: {'chip_factor': 'margin_balance_dollars', 'flow_ts_op': 'ts_delta', 'flow_window': 55, 'noise_ts_op': 'ts_stddev', 'noise_window': 10, 'combine': 'ratio', 'eps': 0.0006511554863310298, 'k': 0.411329196013009, 'cs_norm': 'cs_scale'}. Best is trial 48 with value: 5.279178007123592.


Number of finished trials: 50
Best trial parameters: {'chip_factor': 'foreign_holding_ratio', 'flow_ts_op': 'ts_mean', 'flow_window': 50, 'noise_ts_op': 'ts_stddev', 'noise_window': 10, 'combine': 'ratio', 'eps': 0.0006761346484273701, 'k': 0.223737259341908, 'cs_norm': 'cs_scale'}
Best score: 5.279178007123592


In [74]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_interaction_method,params_pattern,params_價量因子,params_多因子期間參數1,params_多因子期間參數2,params_時序整理1,params_時序整理2,params_財報因子,params_財報籌碼因子,state
0,0,3.155668,2025-12-27 02:15:56.214476,2025-12-27 02:16:24.188393,0 days 00:00:27.973917,div,fin_price_mix,high,15,20,ts_idxmax,ts_mean,cff,NaN,COMPLETE
1,1,1.088151,2025-12-27 02:16:24.189392,2025-12-27 02:16:27.044919,0 days 00:00:02.855527,mul,relative_strength,high,35,45,ts_sum,ts_delay,NaN,non_current_liability,COMPLETE
2,2,1.079456,2025-12-27 02:16:27.044919,2025-12-27 02:16:30.955976,0 days 00:00:03.911057,max,relative_strength,volume,50,45,ts_rev_decay_linear,ts_product,NaN,fcf,COMPLETE
3,3,1.343260,2025-12-27 02:16:30.956975,2025-12-27 02:17:42.676804,0 days 00:01:11.719829,add,relative_strength,cap,15,40,ts_idxmax,ts_argmin,NaN,cash,COMPLETE
4,4,4.402852,2025-12-27 02:17:42.676804,2025-12-27 02:17:53.453321,0 days 00:00:10.776517,add,relative_strength,cap,60,15,ts_decay_linear,ts_min,NaN,sales_yoy,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,12.539726,2025-12-27 02:35:25.060576,2025-12-27 02:35:27.823207,0 days 00:00:02.762631,div,fin_price_mix,open,10,5,ts_min,ts_stddev,ebit,NaN,COMPLETE
96,96,12.539726,2025-12-27 02:35:27.823207,2025-12-27 02:35:30.539564,0 days 00:00:02.716357,div,fin_price_mix,open,10,5,ts_min,ts_stddev,ebit,NaN,COMPLETE
97,97,12.539726,2025-12-27 02:35:30.539564,2025-12-27 02:35:33.278391,0 days 00:00:02.738827,div,fin_price_mix,open,10,5,ts_min,ts_stddev,ebit,NaN,COMPLETE
98,98,12.539726,2025-12-27 02:35:33.278391,2025-12-27 02:35:35.970030,0 days 00:00:02.691639,div,fin_price_mix,open,10,5,ts_min,ts_stddev,ebit,NaN,COMPLETE


In [22]:
expr_list

{'cs_rank(ts_mean(foreign_holding_ratio, 15)/(ts_stddev(foreign_holding_ratio, 25 ) + 4.6144455572754194e-05))': '5.015, 0.790',
 'cs_rank(ts_mean(foreign_holding_ratio, 15)/(ts_stddev(foreign_holding_ratio, 30 ) + 5.063235769717397e-05))': '5.024, 1.018',
 'cs_rank(ts_mean(foreign_holding_ratio, 20)/(ts_stddev(foreign_holding_ratio, 30 ) + 0.0002028562010456259))': '5.117, 1.015',
 'cs_rank(ts_mean(foreign_holding_ratio, 15)/(ts_stddev(foreign_holding_ratio, 25 ) + 3.727781739926848e-05))': '5.009, 0.790',
 'cs_rank(ts_mean(foreign_holding_ratio, 15)/(ts_stddev(foreign_holding_ratio, 30 ) + 0.00017209067071598732))': '5.097, 1.018',
 'cs_scale(ts_mean(foreign_holding_ratio, 30)/(ts_stddev(foreign_holding_ratio, 40 ) + 0.00014539747622639723))': '5.047, 0.930',
 'cs_scale(ts_mean(foreign_holding_ratio, 15)/(ts_stddev(foreign_holding_ratio, 30 ) + 0.00022797548112001983))': '5.125, 1.016',
 'cs_scale(ts_mean(foreign_holding_ratio, 20)/(ts_stddev(foreign_holding_ratio, 35 ) + 0.000220089

In [76]:
current_num = 231
for expr in expr_list:
   # new_key = str(len(alpha_dict) + 1 + added + len(tmp_dict)).zfill(3)
    tmp_dict[str(current_num)] = expr
    current_num += 1

In [77]:
tmp_dict

{'231': 'max(cs_rank(ts_idxmax(end_cash, 15)),cs_rank(ts_rank(cap, 5)))',
 '232': 'sub(cs_rank(ts_min(gross_margin, 60)),cs_rank(ts_product(daily_ret, 60)))',
 '233': 'cs_rank(max(ts_product(net_sales, 35),ts_argmax(cap, 15)))',
 '234': 'cs_rank(max(ts_product(net_sales, 40),ts_argmax(cap, 15)))',
 '235': 'cs_rank(add(ts_product(current_liability, 35),ts_sum(cap, 10)))',
 '236': 'cs_rank(max(ts_product(net_sales, 40),ts_argmax(cap, 20)))',
 '237': 'cs_rank(max(ts_product(total_liability, 45),ts_argmax(cap, 20)))',
 '238': 'cs_rank(max(ts_sum(monthly_sales, 30),ts_delta(value_dollars, 15)))',
 '239': 'cs_rank(max(ts_sum(monthly_sales, 25),ts_delta(value_dollars, 10)))',
 '240': 'cs_rank(sub(ts_sum(monthly_sales, 30),ts_delta(open, 40)))',
 '241': 'cs_rank(sub(ts_sum(cfo, 30),ts_delta(open, 50)))',
 '242': 'cs_rank(div(ts_min(ebit, 15),ts_stddev(close, 25)))',
 '243': 'cs_rank(div(ts_min(ebit, 15),ts_idxmax(close, 25)))',
 '244': 'cs_rank(div(ts_min(ebit, 20),ts_sum(close, 25)))',
 '245'

In [ ]:
for k, v in list(tmp_dict.items())[-20:]:
    print(f'"{k}": "{v}",')

"247": "cs_rank(div(ts_mean(op_income, 15),ts_product(close, 5)))",
"248": "cs_rank(div(ts_mean(monthly_sales, 20),ts_min(daily_ret, 20)))",
"249": "cs_rank(div(ts_decay_linear(monthly_sales, 30),ts_min(daily_ret, 25)))",
"250": "cs_rank(div(ts_max(monthly_sales, 15),ts_min(daily_ret, 20)))",
"251": "cs_rank(div(ts_sum(ar, 35),ts_min(daily_ret, 25)))",
"252": "cs_rank(div(ts_mean(current_asset, 15),ts_mean(close, 15)))",
"253": "cs_rank(div(ts_mean(gross_margin, 15),ts_sum(close, 15)))",
"254": "cs_rank(div(ts_mean(gross_margin, 15),ts_sum(close, 10)))",
"255": "cs_rank(div(ts_rev_decay_linear(gross_margin, 10),ts_sum(close, 10)))",
"256": "cs_rank(div(ts_rev_decay_linear(gross_margin, 5),ts_sum(close, 5)))",
"257": "cs_rank(div(ts_rev_decay_linear(gross_margin, 5),ts_max(close, 10)))",
"258": "cs_rank(max(ts_rev_decay_linear(gross_margin, 5),ts_max(close, 5)))",
"259": "div(cs_rank(ts_rev_decay_linear(current_asset, 5)),cs_rank(ts_sum(close, 5)))",
"260": "cs_rank(div(ts_rev_decay_lin

: 